# Example Calculation
## Client: INTERNAL
---
## Project: Test hydraulic network tools
## Calc: CALC-003, network hydraulic sample calculations
---
## Authentication
> Stamp, Permit
---
## Revision History
|Revision | Date         | Description          | By  | Reviewer|
| :-------| :------------|:---------------------|:----|:--------|
|         |              | DRAFT                | KCD |         |
|    4    |              | no  control valves   | kcd |         |
|    5    | jan 30, 2021 | add control valves   | kcd |         |


---

## Introduction

In [29]:
# dependancies
using Pkg
#Pkg.add("DataFrames")
#Pkg.add("GridInterpolations")
#Pkg.add("Test")

# module is called Hydraulics2
Pkg.add(PackageSpec(url="https://github.com/kevindorma/Hydraulics2.jl"))

using DataFrames
#using GridInterpolations
#using Test
using CSV
using ExcelFiles
using Hydraulics2

#include("/Users/kevin/.julia/dev/Hydraulics/src/Hydraulics2.jl") # my hydraulic libary




  Updating git-repo `https://github.com/kevindorma/Hydraulics2.jl`
  Updating git-repo `https://github.com/kevindorma/Hydraulics2.jl`
 Resolving package versions...
  Updating `~/.julia/environments/v1.3/Project.toml`
 [no changes]
  Updating `~/.julia/environments/v1.3/Manifest.toml`
 [no changes]


In [30]:
# test the global variable containing the list of nominal pipe sizes
Hydraulics2.npsList

,NPS
,Float64
1,0.125
2,0.25
3,0.375
4,0.5
5,0.75
6,1.0
7,1.25
8,1.5
9,2.0


## Define the hydraulic network

Below is the network diagram.

![](./SampleNetworkDiagram.png)

Load the files for defining the network hydraulics.

In [31]:
# these are the specific data
# a few more tables for networks
lineHydraulics = DataFrame(load("networkHydraulicSample_rev_1.xlsx", "hydraulics"))
fluidList = DataFrame(load("networkHydraulicSample_rev_1.xlsx", "fluidList"))
fittingList = DataFrame(load("networkHydraulicSample_rev_1.xlsx", "fittingList"))
nodeList = DataFrame(load("networkHydraulicSample_rev_1.xlsx", "nodeList"))
connectivity = DataFrame(load("networkHydraulicSample_rev_1.xlsx", "connectivity"))
dofList = DataFrame(load("networkHydraulicSample_rev_1.xlsx", "DoF"))

,DoFlabel,objType,object,variable,value
,String,String,Any,String,Float64
1,DOF1,Segment,A,flowRate,15000.0
2,DOF2,Segment,B,flowRate,10000.0
3,DOF3,Node,4.0,pressure,100.0
4,DOF4,Segment,D,flowRate,50000.0
5,DOF5,Node,6.0,pressure,100.0
6,DOF6,Node,7.0,pressure,2000.0
7,DOF7,Node,8.0,pressure,100.0
8,DOF8,Node,9.0,pressure,1000.0
9,DOF9,Segment,F,flowRate,10000.0


## Check input tables for inconsistancies

In [32]:
# show the list of pipe schedules
Hydraulics2.schedList;

In [33]:
 Hydraulics2.checkLineList(lineHydraulics,fluidList)
# error entries are returned
# no entries means good

,message,entry,Segment,NPS,Schedule,material,fluidName
,String,Int64,String,Float64,String,String,String


## Network Hydraulic calculations




In [34]:
# parrot the input data to check
fittingList;
connectivity;
nodeList;


In [35]:
# this does all of the work
gBx = Hydraulics2.doNetworkHydraulics(lineHydraulics, fluidList, connectivity, dofList, nodeList, fittingList)

OK: 36 equations and variables


36×1 Array{Float64,2}:
  15000.0                 
  10000.000000000002      
  24999.999999999996      
  50000.00000000001       
 303968.94252981344       
  10000.0                 
  10000.0                 
  10000.0                 
      2.25e8              
      1.0000000000000001e8
      6.249999999999999e8 
      2.5000000000000005e9
      9.239711802269302e10
      ⋮                   
    100.61914410974748    
    100.42534956480773    
    100.24676864520227    
    100.0                 
    103.82163876298584    
    100.0                 
   2000.0                 
    100.0                 
   1000.0                 
    994.9557305530527     
    104.96747884277313    
    100.0                 

In [36]:
# extract the results from the solution vector.
ourResults = Hydraulics2.extractResults(gBx, lineHydraulics, connectivity, nodeList, fittingList)

,Segment,massFlow,dp,fromP,toP,fromNode,toNode,segmentLength
,Any,Any,Any,Any,Any,Float64,Float64,Float64
1,A,15000.0,0.372375,100.619,100.247,1.0,3.0,100.0
2,B,10000.0,0.178581,100.425,100.247,2.0,3.0,100.0
3,C,25000.0,0.246769,100.247,100.0,3.0,4.0,100.0
4,D,50000.0,3.82164,103.822,100.0,5.0,6.0,100.0
5,E,3.03969e5,1900.0,2000.0,100.0,7.0,8.0,200.0
6,F,10000.0,5.04427,1000.0,994.956,9.0,10.0,100.0
7,CV-001,10000.0,889.988,994.956,104.967,10.0,11.0,0.0
8,G,10000.0,4.96748,104.967,100.0,11.0,12.0,100.0


In [37]:
# combine results into a complete table
allResults = Hydraulics2.combineResults(ourResults,lineHydraulics)

,Segment,massFlow,dp,fromP,toP,fromNode,toNode,segmentLength
,Any,Any,Any,Any,Any,Float64?,Float64?,Float64?
1,A,15000.0,0.372375,100.619,100.247,1.0,3.0,100.0
2,B,10000.0,0.178581,100.425,100.247,2.0,3.0,100.0
3,C,25000.0,0.246769,100.247,100.0,3.0,4.0,100.0
4,D,50000.0,3.82164,103.822,100.0,5.0,6.0,100.0
5,E,3.03969e5,1900.0,2000.0,100.0,7.0,8.0,200.0
6,F,10000.0,5.04427,1000.0,994.956,9.0,10.0,100.0
7,CV-001,10000.0,889.988,994.956,104.967,10.0,11.0,0.0
8,G,10000.0,4.96748,104.967,100.0,11.0,12.0,100.0


In [38]:
# list all of the field names
names(allResults)

24-element Array{String,1}:
 "Segment"      
 "massFlow"     
 "dp"           
 "fromP"        
 "toP"          
 "fromNode"     
 "toNode"       
 "segmentLength"
 "velocity_ms"  
 "rho_kgm3"     
 "Re"           
 "frictF"       
 "IDmm"         
 "IDinch"       
 "Description"  
 "LineTag"      
 "PnID"         
 "NPS"          
 "Schedule"     
 "Material"     
 "fluidName"    
 "inletP_kPaa"  
 "mu_mPas"      
 "roughnessMM"  

In [39]:
# create result tables based on the field names
# overall pressure profile
resultTable1 = select(allResults, [:Segment, :massFlow, :dp, :fromP, :toP])
# piping dimensions
resultTable2 = select(allResults, [:Segment, :NPS, :Schedule, :IDmm, :roughnessMM, :segmentLength])
# friction information
resultTable3 = select(allResults, [:Segment, :fluidName, :velocity_ms, :rho_kgm3, :mu_mPas, :Re, :frictF])

,Segment,fluidName,velocity_ms,rho_kgm3,mu_mPas,Re,frictF
,Any,String?,Float64?,Float64?,Float64?,Float64?,Float64?
1,A,water,0.224756,993.99,0.719,47881.5,0.0222404
2,B,water,0.149837,993.99,0.719,31921.0,0.0240212
3,C,water,0.2165,993.99,0.719,60668.7,0.0209884
4,D,water,0.749187,993.99,0.719,1.59605e5,0.0187372
5,E,water,10.3348,993.99,0.719,1.46161e6,0.018103
6,F,water,0.586342,993.99,0.719,63145.3,0.0226475
7,CV-001,missing,missing,missing,missing,missing,missing
8,G,water,0.586342,993.99,0.719,63145.3,0.0226475


In [40]:
# export to CSV file if needed